In [0]:
rawdata_path_Historical = "abfss://rawdata@datasavergenv2.dfs.core.windows.net/Bike_Historical_Raw_Data/Bike_Histroical_data1/"
df_Historical = spark.read.format("parquet").load(rawdata_path_Historical)
old_rawdata_path = "abfss://rawdata@datasavergenv2.dfs.core.windows.net/Bike_Raw_Data/Bike_Raw_Data/"
old_df_bike = spark.read.format("parquet").load(rawdata_path)



In [0]:
merge_condition = """
old.id = new.id
AND old.timestamp = new.timestamp
"""
df_bike = old_df_bike.alias("old").merge(
    df_Historical.alias("new"),
    merge_condition
    ).whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS data_transformation.bronze;


In [0]:
%sql
ALTER TABLE data_transformation.bronze.bronze_historical_bike 
SET TBLPROPERTIES ('delta.feature.timestampNtz' = 'supported');

In [0]:
df_Historical.write \
    .format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable("data_transformation.bronze.bronze_Historical_Bike")


In [0]:
df_Historical.columns 

In [0]:
%sql
SELECT * FROM data_transformation.bronze.bronze_historical_bike ORDER BY timestamp ASC ;